In [2]:
from dotenv import load_dotenv
from pathlib import Path
import os
import sys
import json
import requests
import openai

env_path = Path('../..') / '.env'
load_dotenv(dotenv_path=env_path)

print(os.getcwd())
sys.path.append('')

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
        

/Users/davidwei/Documents/SellScale/sellscale-api/notebooks/linkedin_conversation


In [18]:
def detect_scheduling(conversation_history: list[str]) -> bool:
    """
    Detects if a conversation is about scheduling a call.
    """
    prompt_template = "conversation thread:\n"
    for message in conversation_history:
        prompt_template += f"{message}\n"

    prompt = "Is the person trying to schedule a time to meet with me? Answer with 'Yes' or 'No'."

    full_prompt = prompt_template + "\n\n" + prompt + "\n\ncompletion:"

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=full_prompt,
        max_tokens=5,
        temperature=0,
    )

    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return []

    choices = response["choices"]
    top_choice = choices[0]
    text: str = top_choice["text"].lower().strip()

    return True if text == "yes" else False


In [19]:
conversation_history = [
    "me: Hi Jim, I'm a software engineer at SellScale. I'm reaching out to you because I saw that you're a software engineer at Google. I'm looking to connect with other software engineers to learn more about the industry. Would you be open to a quick 15 minute call to learn more about your experience at Google?",
    "me: Just wanted to follow up on my previous message. Would you be open to a quick 15 minute call to learn more about your experience at Google?",
    "me: I don't want to take up too much of your time, but I'm curious to learn more about your experience at Google. Would you be open to a quick 15 minute call?",
    "them: Sure, I'd be happy to chat. What's your availability like?",
]
print(detect_scheduling(conversation_history))

conversation_no_scheduling = [
    "me: Hi Jeremey, I'm a software engineer at SellScale. I'm reaching out to you because I saw that you're a software engineer at Google. I'm looking to connect with other software engineers to learn more about the industry. Would you be open to a quick 15 minute call to learn more about your experience at Google?",
    "me: Just wanted to follow up on my previous message. Would you be open to a quick 15 minute call to learn more about your experience at Google?",
    "me: I don't want to take up too much of your time, but I'm curious to learn more about your experience at Google. Would you be open to a quick 15 minute call?",
    "them: Sorry I'm too busy, circle back in a few months."
]
print(detect_scheduling(conversation_no_scheduling))

True
False
